In [1]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd

# 設定 data_path
dir_data = './data/'

In [2]:
f_app_train = os.path.join(dir_data, 'application_train.csv')
app_train = pd.read_csv(f_app_train)

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

## 練習時間

觀察有興趣的欄位的資料分佈，並嘗試找出有趣的訊息
#### Eg
- 計算任意欄位的平均數及標準差
- 畫出任意欄位的[直方圖](https://zh.wikipedia.org/zh-tw/%E7%9B%B4%E6%96%B9%E5%9B%BE)

### Hints:
- [Descriptive Statistics For pandas Dataframe](https://chrisalbon.com/python/data_wrangling/pandas_dataframe_descriptive_stats/)
- [pandas 中的繪圖函數](https://amaozhao.gitbooks.io/pandas-notebook/content/pandas%E4%B8%AD%E7%9A%84%E7%BB%98%E5%9B%BE%E5%87%BD%E6%95%B0.html)


In [4]:
# 列出刪除空值前的行列數
(rows1, cols1) = app_train.shape
print(f'rows1 = {rows1}, cols1 = {cols1}')

# 刪除空值(只要每列有出現1個空值就刪除該列)
app_train2 = app_train.dropna(axis='index', how='any')

# 重設index
app_train2 = app_train2.reset_index()

# 列出刪除空值後的行列數
(rows2, cols2) = app_train2.shape
print(f'rows2 = {rows2}, cols2 = {cols2}')

# 隨便取出10個不重複的欄位
col_rand = np.random.choice(list(app_train2.head(0)), size=10, replace=False)
app_train3 = app_train2.loc[:, col_rand]

# 只保留非字串欄位
for column in app_train3.columns:
    value = app_train3.at[0, column]
    if (type(value) is str):
        try:
            pd.to_numeric(value)
        except ValueError:
            app_train3 = app_train3.drop(column, axis='columns')
            pass
(rows3, cols3) = app_train3.shape
print(f'rows3 = {rows3}, cols3 = {cols3}')
#print(app_train3.head(10))

# 建立資料量化輸出表格
name = app_train3.columns
subject = ['Mean','Median', 'Mode', 'Min', 'Max', 'Range', 'Quartiles', 'Variance', 'S.D.']
df = pd.DataFrame(index=name, columns=subject)

# 每個欄位各自計算
for column in app_train3.columns:
    # 平均值
    df['Mean'].loc[column] = app_train3.loc[:, column].mean()
    # 中位數
    df['Median'].loc[column] = app_train3.loc[:, column].median()
#     # 眾數
#     df['Mode'].loc[column] = app_train3.loc[:, column].mode()
    # 最⼩值
    df['Min'].loc[column] = app_train3.loc[:, column].min()
    # 最⼤值
    df['Max'].loc[column] = app_train3.loc[:, column].max()
#     # 範圍
#     df['Range'].loc[column] = app_train3.loc[:, column].range()
    # 四分位差(10%)
    df['Quartiles'].loc[column] = app_train3.loc[:, column].quantile(0.1)
#     # 變異數
#     df['Variance'].loc[column] = app_train3.loc[:, column].var()
#     # 標準差
#     df['S.D.'].loc[column] = app_train3.loc[:, column].std()    

print(df)
#continue

rows1 = 307511, cols1 = 122
rows2 = 8602, cols2 = 123
rows3 = 8602, cols3 = 9
                               Mean  Median Mode    Min Max Range Quartiles  \
NONLIVINGAPARTMENTS_AVG  0.00923785       0  NaN      0   1   NaN         0   
FLAG_DOCUMENT_2                   0       0  NaN      0   0   NaN         0   
ENTRANCES_MEDI              0.15467  0.1379  NaN      0   1   NaN     0.069   
FLOORSMIN_MEDI             0.242391  0.2083  NaN      0   1   NaN    0.0417   
REGION_RATING_CLIENT        2.00488       2  NaN      1   3   NaN         1   
OWN_CAR_AGE                 11.2241       8  NaN      0  65   NaN         2   
CNT_FAM_MEMBERS             2.40142       2  NaN      1   7   NaN         1   
DAYS_ID_PUBLISH            -2975.93   -3197  NaN  -6551  -1   NaN     -4735   
LIVINGAPARTMENTS_MEDI      0.105958   0.077  NaN      0   1   NaN    0.0257   

                        Variance S.D.  
NONLIVINGAPARTMENTS_AVG      NaN  NaN  
FLAG_DOCUMENT_2              NaN  NaN  
ENTRANCES_M